## Initalize notebook and load configuration

In [2]:
import esf_utils
import os 
import pandas as pd 
import xml.etree.ElementTree as ET
import shutil
import configparser
import time
import datetime
import sys

print ( 'dependencies loaded.')

campaign_esf_paths_file = 'turn_one.txt'
campaign_files = esf_utils.parse_campaign_files_txt( campaign_esf_paths_file )


config = configparser.ConfigParser()
config.read('config.ini')

esf2xml_dir =  config['dependencies']['esf2xml']
save_folder = config['paths']['save_game_folder']
output_folder = config['paths']['output_folder']
extracted_subfolder = config['paths']['extracted_subfolder']
cwd = os.getcwd()

print('notebook initialized')

dependencies loaded.
loading esf paths from file @  turn_one.txt
notebook initialized


## Testing all together 

In [18]:
# notebook for parsing a file 
econ_array = []
army_array = [] 
region_array = []

max = len(campaign_files)-1
i = 0 

for s in campaign_files:

    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    
    full_path = os.path.join( save_folder , s )
    #output_dir =  f"{save_file_clean}_extract"
    file_modstamp = os.path.getmtime(full_path)
    ts = datetime.datetime.fromtimestamp( file_modstamp )
    unix_timestamp = int(time.mktime(ts.timetuple()))

    try:
        dat1 = esf_utils.extract_save_file( 
        save_folder 
        , s
        ,templated
        ,config 
        )
    except:
        print("An exception occurred while extracting")
    

    extracted_output = os.path.join(templated, extracted_subfolder)
    
    try :
        session_id = esf_utils.get_session_guid( extracted_output )[1]
        turn_num = esf_utils.get_turn_number( extracted_output )
    
        # for faction economics at a high KPI level
        new_array = []
        esf_utils.parse_extracted_factions_folder( extracted_output, new_array ) 
        for r in new_array:
            r["session_id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            econ_array.append( r ) 
        econ_df = pd.DataFrame( econ_array )
        econ_df.to_csv('export_faction_economy.csv')
        
        # parse army information
        new_array = []
        #session_id, session_guid, turn_num = 
        esf_utils.parse_extracted_armies_folder( extracted_output, new_array ) 

        # write in all data universal to this save file
        for r in new_array:
            r["session_id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            army_array.append( r )
        army_df = pd.DataFrame( army_array )
        army_df.to_csv('export_army_unit.csv')

        # parse region information
        new_array = []
        esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
        
         # write in all data universal to this save file
        for r in new_array:
            r["session_id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            region_array.append( r )
        region_df = pd.DataFrame( region_array ) 
        region_df.to_csv( 'export_region.csv' )
        
        # first run - get province hierarchy 
        if i == 0 :
            print ( 'parsing province xml data')
            new_array = []
            esf_utils.parse_province_campaign_data( extracted_output , new_array )
            province_df = pd.DataFrame( new_array )
            province_df.to_csv( 'export_province.csv')
    except:
        print("An exception occurred while extracting")
   
    print( f"@ { i }  / { max } folders loaded and exported")
    i += 1

skipping extract...
getting economy data from  extract\Clan_Skryre_Auto_save.181597358761.save_extract\extract\
An exception occurred while extracting
@ 0  / 1 folders loaded and exported
need to convert to .esf for esf2xml to work, copying file....
 copy D:\TW2_SAVEGAME_BACKUPS\Clan Skryre_Auto-save.191597358761.save -> c:\lab\tw_save\Clan_Skryre_Auto_save.191597358761.esf
jruby C:\lab\etwng-master\esfxml\esf2xml c:\lab\tw_save\Clan_Skryre_Auto_save.191597358761.esf c:\lab\tw_save\extract\Clan_Skryre_Auto_save.191597358761.save_extract\
jruby C:\lab\etwng-master\esfxml\esf2xml c:\lab\tw_save\extract\Clan_Skryre_Auto_save.191597358761.save_extract\compressed_data.esf c:\lab\tw_save\extract\Clan_Skryre_Auto_save.191597358761.save_extract\extract\


# Just testing extract_regions  

In [ ]:
for s in campaign_files:
    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    full_path = os.path.join( save_folder , s )   
    extracted_output = os.path.join(templated, extracted_subfolder)

    # for region and control 
    new_array = []
    esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
    region_df = pd.DataFrame( new_array )
    region_df.to_csv( 'export_region.csv' , index=False )
    print ( 'exported to region.csv')   
 

In [ ]:
## Just provinces 


def parse_province_campaign_data( extracted_output, new_array ):
    full_path = os.path.join( extracted_output , 'campaign_env' , 'world-0000.xml' ) 
    xml = ET.parse( full_path ).getroot()    

    # get name NK of region from file, NOT filename 
    province_xml = xml.findall( "./rec/ary/rec[@type = 'PROVINCE_ARRAY']")

    xml_index = 0 
    for province in province_xml: 

        #if xml_index == 0 :
        # parse for testing 
        province_name = province.findall( './asc')[0].text 
        #print ( 'province name is ' , province_name ) 

        region_blocks = province.findall( "./rec/rec/ary/rec/asc" )
        #print ( 'length of region blocks ', len( region_blocks ) )
        for region in region_blocks : 
            data_row = {
                'province_name' : province_name
                ,'settlement_key' : region.text
            }
            new_array.append( data_row )
    xml_index = xml_index + 1

for s in campaign_files:
    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    full_path = os.path.join( save_folder , s )   
    extracted_output = os.path.join(templated, extracted_subfolder)

    # for characters
    new_array = []
    esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
    region_df = pd.DataFrame( new_array )
    region_df.to_csv( 'export_region.csv' , index=False )
    print ( 'exported to region.csv')   
 

In [12]:
## Just character 


def parse_character_campaign_data( extracted_output, new_array ):

    print( 'getting character data from ', extracted_output )

    #print ( 'debugger')
    # get factions data
    character_dir = os.path.join( extracted_output , "character" )


    arr = os.listdir( character_dir )
    for file in os.listdir( character_dir ):
        #print('file', file )
        if file.endswith(".xml"):
            if file.find("general") != -1:
                # parse_faction_xml( factions_dir , file, data_array )
                
                full_path = os.path.join( character_dir , file ) 
                print(full_path )
                xml = ET.parse( full_path ).getroot()
                
                coord_xpath = "./rec/rec[@type = 'LOCOMOTABLE']/v2"   
                coords = xml.findall( coord_xpath )[0]

                x = coords.attrib['x']
                y = coords.attrib['y']
                print( coords , x ,y ) 
                
                details_xpath = "./rec/rec[@type = 'CHARACTER_DETAILS']/asc"   
                detail_tags = xml.findall( details_xpath )
                faction = detail_tags[0].text 
                type = detail_tags[2].text 
                key = detail_tags[3].text

                data_row = {
                    'faction' : faction
                    ,'type' : type
                    ,'key' : key 
                    , 'loc.x' : x
                    , 'loc.y' : y  
                }
                new_array.append( data_row )
                print('debugger')

                '''
                campaign_tags = mf_xml.findall( "./rec/rec/[@type = 'LOCOMOTABLE']" )[0] 
                localized = campaign_tags.findall( "asc")
                unit_campaign_name = localized[0].text

                if ( unit_campaign_name is None ):
                    unit_campaign_name = 'null'

                '''

for s in campaign_files:
    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    full_path = os.path.join( save_folder , s )   
    extracted_output = os.path.join(templated, extracted_subfolder)

    # for characters
    new_array = []
    parse_character_campaign_data( extracted_output , new_array )

    
    #esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
    characters_df = pd.DataFrame( new_array )
    region_df.to_csv( 'export_region.csv' , index=False )
    print ( 'exported to region.csv')   
 

getting character data from  extract\turn1_eatine_noMove.1349987008.save_extract\extract\
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0001.xml
<Element 'v2' at 0x0000023E4B9A4680> 140.37932 211.49678
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0002.xml
<Element 'v2' at 0x0000023E4B90FEA0> 158.42809 62.908714294433594
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0003.xml
<Element 'v2' at 0x0000023E4B9C4C20> 151.07489 278.65088
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0004.xml
<Element 'v2' at 0x0000023E4B9C4810> 64.1734 302.57935
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0005.xml
<Element 'v2' at 0x0000023E4B9D6860> 193.85715 265.52881
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0006.xml
<Element 'v2' at 0x0000023E4B9C1680> 187.8409 268.2

In [20]:
import xml.etree.ElementTree as ET
xml_path = "./extract/Clan_Skryre_Auto_save.171597358761.save_extract/extract/army/ARMY-0001.xml"

xml  = ET.parse( xml_path ).getroot()

mf_legacy = "./rec/rec/rec/[@type = 'MILITARY_FORCE_LEGACY_HISTORY']"
mf_legacy_xml = xml.find( mf_legacy )
char_path = "./rec/ary/rec/rec/asc"
name_tags = mf_legacy_xml.findall( char_path )
tags_list = []
for tag in name_tags : 
    if tag.text != None :
        tags_list.append( tag.text ) 
name_nk = '_'.join(tags_list)

#mf_xpath = "./rec/[@type = 'MILITARY_FORCE']"   
#mf_xml = xml.findall( mf_xpath )[0]
#name_blocks = "./rec/rec[@type = 'CHARACTER_DETAILS']/"  
'''
  units_arr_xpath = "./rec/ary/rec/rec/[@type = 'UNIT']"
    units_xml = mf_xml.findall( units_arr_xpath )

    faction_name_xpath = "./rec/[@type='COMMANDER_DETAILS']/asc"
'''


print( 'debugger')

In [13]:
import xml.etree.ElementTree as ET
from lxml import etree
# turn 3 
xml_path = "./extract/ikit_claw_turn1_battle_ar.13389809406.save_extract/extract/campaign_env/campaign_model-0000.xml"
#"./extract/Clan_Skryre_Auto_save.31597358761.save_extract/extract/campaign_env/campaign_model-0000.xml"
_parser = etree.XMLParser(recover=True)
xml = ET.parse( xml_path , parser = _parser )
br_xpath = "./rec/rec/[@type = 'BATTLE_RESULTS']/"
br_tags = xml.findall( br_xpath )


ally_xpath =  "./rec/rec/rec/ary/[@type='ALLIANCES']"
ally_tags = xml.findall( ally_xpath )

xpath =       "./rec/rec/rec/ary/[@type='ALLIANCES']/rec/rec/ary/rec/rec/rec"

bsf_tags = xml.findall( xpath )

ally_xpath =  "./rec/rec/rec/ary/[@type='ALLIANCES']"
ally_tags = xml.findall( ally_xpath )
#battle_result_xpath = "./rec/rec/rec/ary/[@type='ALLIANCES']/rec"
battle_result_xpath = "./rec/rec/rec/ary/rec/[@type='BATTLE_RESULT_ALLIANCE']/"
brx = xml.findall( battle_result_xpath  )
# direct past 
bra = "./rec/rec/rec/ary/rec/[@type ='BATTLE_RESULT_ALLIANCE']/"
bra_tags = xml.findall( battle_result_xpath )
'''
<rec type="BATTLE_RESULTS" version="2">
   <yes/>
   <i>0</i>
   <u>615</u>
   <flt>-1.0</flt>
   <ary type="ALLIANCES">
    <rec type="ALLIANCES">
     <rec type="BATTLE_RESULT_ALLIANCE" version="7">
'''
#for t in bsf_tags : 
asc_tags = xml.findall( f"{xpath}/asc")
faction = asc_tags[0].text  

flt_tags = xml.findall( f"{xpath}/flt")

json = {} 
for i in range ( 0 , 21 ):
  json[ f"bsa_float_{i}"] = flt_tags[i].text

print( 'debugger')

for t in br_tags : 
  ''' <yes/>
   <i>0</i>
   <u>366</u>
   <flt>-1.0</flt>'''
  

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1949, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1984, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [45]:
import xml.etree.ElementTree as ET
from lxml import etree
# turn 3 
xml_path = "./extract/ikit_claw_turn1_battle_ar.13389809406.save_extract/extract/campaign_env/campaign_model-0000.xml"
_parser = etree.XMLParser(recover=True)
xml = ET.parse( xml_path , parser = _parser )
br_xpath = "./rec/rec/[@type = 'BATTLE_RESULTS']"
br_tags = xml.findall( br_xpath )

xpath = f"{br_xpath}/ary/[ @type = 'ALLIANCES']/rec/"
xtags = xml.findall( xpath )

# setup battle stats
json = {}
for tag in xtags:
   battle_data = tag.findall( 'u' )
   if len( battle_data ) >= 6 : 
      json[ 'deployed' ] = battle_data[0].text
      json[ 'remaining' ] = battle_data[2].text
      json[ 'lost' ] = battle_data[4].text
      json[ 'killed' ] = battle_data[6].text

# f"{br_xpath}/ary/[ @type = 'ALLIANCES']/rec/rec/ary/rec/rec/rec" #[ @type = 'BATTLE_RESULT_UNIT' ]/"

# fun fact - ending in / gets all children, not / gets all tags 
army_units_xpath1 = f"{br_xpath}/ary/[ @type = 'ALLIANCES']/rec/rec/ary/rec/rec/ary[ @type = 'UNITS']/rec[ @type = 'UNITS']/rec[ @type = 'BATTLE_RESULT_UNIT']"
army_units_xpath2 = f"{army_units_xpath1}"
army_units_tags = xml.findall( army_units_xpath2 )

unit_result_array = [] 
for t in army_units_tags : 
   unit_data = t.findall( 'u') 

   json = {} 
   if len( unit_data ) >= 6 :
      json[ 'kills'] = unit_data[4].text
      json[ 'start_num_units'] = unit_data[0].text
      json[ 'end_num_units'] = unit_data[1].text
      unit_result_array.append ( json ) 

   unit_string_data = t.findall( 's') 
   if len( unit_data ) >= 1 :
      json[ 'unit_name'] = unit_string_data[0].text 

   unit_result_array.append( json )

   print( 'debugger')

# u[5] = kills
# u[0] = start num units
# u[1] = end num units

print( 'debugger')

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1949, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1984, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [60]:
import xml.etree.ElementTree as ET
from lxml import etree
import copy 
# turn 3 
xml_path = "./extract/ikit_claw_turn1_battle_ar.13389809406.save_extract/extract/campaign_env/campaign_model-0000.xml"
_parser = etree.XMLParser(recover=True)
xml = ET.parse( path , parser = _parser )

br_xpath = "./rec/rec/[@type = 'BATTLE_RESULTS']"
# fun fact - ending in / gets all children, not / gets all tags 
#army_units_xpath1 = f"{br_xpath}/ary/[ @type = 'ALLIANCES']/rec/rec/ary/rec/rec/ary[ @type = 'UNITS']/rec[ @type = 'UNITS']/rec[ @type = 'BATTLE_RESULT_UNIT']
br_army_xpath = f"{br_xpath}/ary/[ @type = 'ALLIANCES']/rec/rec/ary/rec/rec[ @type = 'BATTLE_RESULT_ARMY']"
br_army_tags = xml.findall( br_army_xpath )

#new_array = [] 
# Iterate within each army - link these together somehow 
for army in br_army_tags:

    army_json = {} 
    # BATTLE_SETUP_FACTION
    faction_xpath = f"./rec[ @type='BATTLE_SETUP_FACTION']/asc"
    faction_tags = army.findall( faction_xpath )
    faction_nk = faction_tags[0].text
    
    army_attrib_xpath =  f"./s"
    army_attrib_tags = army.findall( army_attrib_xpath )

    ui_path_xpath = f"./asc"
    ui_path_tags = army.findall( ui_path_xpath )

    army_json[ 'leader'] = army_attrib_tags[0].text 
    army_json[ 'ui_path'] =  ui_path_tags[0].text
    army_json[ 'faction_nk'] = faction_nk

    # now parse individual units
    bru_xpath = "./ary[ @type = 'UNITS']/rec[ @type = 'UNITS']/rec[ @type = 'BATTLE_RESULT_UNIT']"
    bru_tags = army.findall( bru_xpath )

    for t in bru_tags : 
        unit_data = t.findall( 'u') 

        json = copy.deepcopy(army_json) 
        if len( unit_data ) >= 6 :
            json[ 'kills'] = unit_data[4].text
            json[ 'start_num_units'] = unit_data[0].text
            json[ 'end_num_units'] = unit_data[1].text

        unit_string_data = t.findall( 's') 
        if len( unit_data ) >= 1 :
            json[ 'unit_name'] = unit_string_data[0].text 

        new_array.append ( json ) 

        print( 'debuger')

    print( 'debuger')

debuger
debuger


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1949, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1984, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
## added 2021/12/15
## Diplomacy per faction 
def parse_diplomacy_from_factions_folder( extracted_output, data_array ):


    factions_dir = os.path.join( extracted_output , "factions" )
    arr = os.listdir( factions_dir )
    for file in os.listdir( factions_dir ):
        if file.endswith(".xml"):
            full_path = os.path.join( factions_dir , file )
            full_path = "C:/lab/tw_save/extract/Clan_Skryre_attack_2.1327371162.save_extract/extract/factions/wh2_main_skv_clan_skyre.xml"
            tree = ET.parse( full_path )
            root = tree.getroot()
            row = {} 
            #xpath = f"./rec/rec/ary/rec/[@type='OLD_DIPLOMACY_RELATIONSHIP']"rec[ @type = 'BATTLE_RESULT_ARMY']
            xpath = f"./rec/[ @type = 'OLD_DIPLOMACY_MANAGER']/ary/rec[ @type='DIPLOMACY_RELATIONSHIPS_ARRAY']"        
            tags = root.findall( xpath )

            for tag in tags :

                u_tags = tag.findall( f"./rec/u" )    
                for x in range ( 0 , len( u_tags )):
                    row[ f"u_{x}"] = u_tags[x].text 

                i_tags = tag.findall( f"./rec/i" )    
                for x in range ( 0 , len( i_tags )):
                    row[ f"i_{x}"] = i_tags[x].text 

                asc_tags = tag.findall( f"./rec/asc" )    
                for x in range ( 0 , len( asc_tags )):
                    row[ f"asc_{x}"] = asc_tags[x].text 
                    
                #tags = root.findall( xpath ) 
                faction_data = esf_utils.parse_faction_metadata( root )
                row['faction_name'] = faction_data['faction_name']
                row['faction_id'] = faction_data['faction_id']
                data_array.append( row )


for s in campaign_files:
    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    full_path = os.path.join( save_folder , s )   
    extracted_output = os.path.join(templated, extracted_subfolder)

    # for characters
    new_array = []
    parse_diplomacy_from_factions_folder( extracted_output, new_array ) 
    region_df = pd.DataFrame( new_array )
    region_df.to_csv( 'export_diplomacy.csv' , index=False )
    print ( 'exported to region.csv')   
 

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1949, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\benmc\.conda\envs\tw_save\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1984, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 